In [2]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import time

import torch
import torch.nn as nn
import torch.nn.functional as functions
import torch.optim as optimize

import torchvision
import torchvision.transforms as transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [4]:
train_data = torchvision.datasets.CIFAR10(root = "./dataset", train = True, transform= transform, download=True)
test_data = torchvision.datasets.CIFAR10(root = "./dataset", train = False, transform= transform, download=True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=True)

In [5]:
class_names = ['airplane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [6]:
class NeuralNetwork(nn.Module):

  def __init__(self):
    super().__init__()

    def conv_block(in_channels, out_channels, pool=False):
        layers = [
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding = 1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        ]
        if pool:
            layers.append(nn.MaxPool2d(2))
        return nn.Sequential(*layers)
        
    self.layer1 = conv_block(3,64)     
    self.layer2 = conv_block(64, 64, pool=True)

    self.layer3 = conv_block(64, 128)
    self.layer4 = conv_block(128, 128, pool=True)

    self.layer5 = conv_block(128, 256)
    self.layer6 = conv_block(256, 256, pool=True)

    self.gap = nn.AdaptiveAvgPool2d(1)
    self.flatten = nn.Flatten()
    self.fc = nn.Linear(256,10)

    self.dropout = nn.Dropout(0.5)

  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
      
    x = self.layer3(x)
    x = self.layer4(x)
      
    x = self.layer5(x)
    x = self.layer6(x)

    x = self.gap(x)
    x = self.flatten(x)
    x = self.dropout(x)
    x = self.fc(x)
    return x

In [7]:
network = NeuralNetwork().to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optimize.SGD(network.parameters(), lr=0.1, weight_decay=1e-4, momentum=0.9)

In [7]:
train_losses = []
test_losses = []
train_accuracies = []
test_accuracies = []

epochs = 20

print("Started training...")

for epoch in range(epochs):
    start_time = time.time()

    network.train() 
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device) 

        optimizer.zero_grad()
        outputs = network(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

        epoch_train_loss = running_loss / len(train_loader)
        epoch_train_acc = 100 * correct_train / total_train
    
        train_losses.append(epoch_train_loss)
        train_accuracies.append(epoch_train_acc)

        end_time = time.time()
        epoch_duration = end_time - start_time

    print(f"Epoch: {epoch+1}/{epochs} | "
          f"Train loss: {epoch_train_loss:.3f} | "
          f"Train Accuracy: {epoch_train_acc:.3f}% | "
          f"Time: {epoch_duration:.3f} sec|")

print("Finished Training")

Started training...
Epoch: 1/20 | Train loss: 1.513 | Train Accuracy: 44.890% | Time: 224.463 sec|
Epoch: 2/20 | Train loss: 1.018 | Train Accuracy: 63.652% | Time: 229.091 sec|
Epoch: 3/20 | Train loss: 0.812 | Train Accuracy: 71.318% | Time: 222.894 sec|
Epoch: 4/20 | Train loss: 0.674 | Train Accuracy: 76.608% | Time: 228.952 sec|
Epoch: 5/20 | Train loss: 0.579 | Train Accuracy: 79.830% | Time: 231.453 sec|
Epoch: 6/20 | Train loss: 0.504 | Train Accuracy: 82.626% | Time: 225.305 sec|
Epoch: 7/20 | Train loss: 0.442 | Train Accuracy: 84.924% | Time: 232.351 sec|
Epoch: 8/20 | Train loss: 0.388 | Train Accuracy: 86.680% | Time: 231.368 sec|
Epoch: 9/20 | Train loss: 0.348 | Train Accuracy: 87.958% | Time: 223.167 sec|
Epoch: 10/20 | Train loss: 0.306 | Train Accuracy: 89.480% | Time: 228.034 sec|
Epoch: 11/20 | Train loss: 0.278 | Train Accuracy: 90.470% | Time: 224.229 sec|
Epoch: 12/20 | Train loss: 0.245 | Train Accuracy: 91.528% | Time: 225.244 sec|
Epoch: 13/20 | Train loss: 0.

In [8]:
torch.save(network.state_dict(), "./model.pth")
network = NeuralNetwork()
network.load_state_dict(torch.load("./model.pth"))

<All keys matched successfully>

In [12]:
test_loss = 0.0
correct = 0
total = 0

network.eval()
with torch.no_grad():
  for data in test_loader:
    images, labels = data
    images, labels = images.to(device), labels.to(device)
                                                  
    outputs = network(images)
    loss = loss_function(outputs, labels)
    test_loss += loss
      
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

final_test_acc = 100 * correct / total
final_test_loss = test_loss / len(test_loader)
print(f"Accuracy: {final_test_acc:.3f}%")
print(f"Loss: {final_test_loss:.3f}%")

Accuracy: 83.540%
Loss: 0.635%
